In [8]:
import pandas as pd

# https://files.grouplens.org/datasets/movielens/ml-25m.zip
movies = pd.read_csv("movies.csv")

In [9]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [10]:
import re

def clean_title(title):
    title = re.sub("[^a-zA-Z0-9 ]", "", title)
    return title

In [11]:
movies["clean_title"] = movies["title"].apply(clean_title)

In [12]:
movies

,movieId,title,genres,clean_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji 1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men 1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale 1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II 1995
...,...,...,...,...
62418,209157,We (2018),Drama,We 2018
62419,209159,Window of the Soul (2001),Documentary,Window of the Soul 2001
62420,209163,Bad Poems (2018),Comedy|Drama,Bad Poems 2018
62421,209169,A Girl Thing (2001),(no genres listed),A Girl Thing 2001


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))

tfidf = vectorizer.fit_transform(movies["clean_title"])

In [14]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def search(title):
    title = "Toy Story 1995"
    title = clean_title(title)
    query_vec = vectorizer.transform([title])
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    indices = np.argpartition(similarity, -5)[-5:]
    results = movies.iloc[indices].iloc[::-1]

    return results

In [15]:
import ipywidgets as widgets
from IPython.display import display

movie_input = widgets.Text(
    value='Toy Story',
    description='Movie Title:',
    disabled=False
)
movie_list = widgets.Output()

def on_type(data):
    with movie_list:
        movie_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            display(search(title))

movie_input.observe(on_type, names='value')


display(movie_input, movie_list)

Text(value='Toy Story', description='Movie Title:')

Output()

In [16]:
movie_id = 89745

#def find_similar_movies(movie_id):
movie = movies[movies["movieId"] == movie_id]

In [17]:
ratings = pd.read_csv("ratings.csv")

In [18]:
ratings

,userId,movieId,rating,timestamp
0,1,296,5.0,1.147880e+09
1,1,306,3.5,1.147869e+09
2,1,307,5.0,1.147869e+09
3,1,665,5.0,1.147879e+09
4,1,899,3.5,1.147869e+09
...,...,...,...,...
372357,2559,45720,3.0,1.556488e+09
372358,2559,46578,3.5,1.556850e+09
372359,2559,46970,2.5,1.556850e+09
372360,2559,46972,3.0,1.567280e+09


In [19]:
ratings.dtypes

userId         int64
movieId        int64
rating       float64
timestamp    float64
dtype: object

In [20]:
similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()

In [21]:
similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]

In [22]:
similar_user_recs

3741        318
3742        527
3743        541
3744        589
3745        741
          ...  
369780    78499
369781    81845
369783    88125
369784    89745
369785    93840
Name: movieId, Length: 9033, dtype: int64

In [23]:
similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

similar_user_recs = similar_user_recs[similar_user_recs > .10]

In [24]:
all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]

In [25]:
all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())

In [26]:
all_users

,userId,movieId,rating,timestamp
0,1,296,5.0,1.147880e+09
23,1,3949,5.0,1.147869e+09
48,1,7361,5.0,1.147880e+09
72,2,110,5.0,1.141417e+09
76,2,260,5.0,1.141417e+09
...,...,...,...,...
372224,2558,588,5.0,8.454335e+08
372229,2558,595,5.0,8.454335e+08
372253,2559,1265,5.0,1.556850e+09
372255,2559,1270,5.0,1.556593e+09


In [27]:
rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
rec_percentages.columns = ["similar", "all"]

In [28]:
rec_percentages

,similar,all
movieId,,
89745,1.000000,0.045903
58559,0.588785,0.136422
79132,0.542056,0.128700
2571,0.514019,0.244959
59315,0.485981,0.053625
...,...,...
7143,0.102804,0.021879
59369,0.102804,0.018447
7254,0.102804,0.024024


In [29]:
rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]

In [30]:
rec_percentages = rec_percentages.sort_values("score", ascending=False)

In [31]:
rec_percentages

,similar,all,score
movieId,,,
89745,1.000000,0.045903,21.785047
86332,0.168224,0.010725,15.685234
102125,0.177570,0.012870,13.797196
122914,0.112150,0.008151,13.758977
111362,0.261682,0.019305,13.555140
...,...,...,...
110,0.158879,0.160446,0.990229
593,0.214953,0.227370,0.945389
527,0.205607,0.223509,0.919906


In [32]:
rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")

,similar,all,score,movieId,title,genres,clean_title
17067,1.000000,0.045903,21.785047,89745,"Avengers, The (2012)",Action|Adventure|Sci-Fi|IMAX,Avengers The 2012
16312,0.168224,0.010725,15.685234,86332,Thor (2011),Action|Adventure|Drama|Fantasy|IMAX,Thor 2011
19678,0.177570,0.012870,13.797196,102125,Iron Man 3 (2013),Action|Sci-Fi|Thriller|IMAX,Iron Man 3 2013
25068,0.112150,0.008151,13.758977,122914,Avengers: Infinity War - Part II (2019),Action|Adventure|Sci-Fi,Avengers Infinity War Part II 2019
21606,0.261682,0.019305,13.555140,111362,X-Men: Days of Future Past (2014),Action|Adventure|Sci-Fi,XMen Days of Future Past 2014
21348,0.271028,0.020163,13.441837,110102,Captain America: The Winter Soldier (2014),Action|Adventure|Sci-Fi|IMAX,Captain America The Winter Soldier 2014
16725,0.177570,0.013299,13.352125,88140,Captain America: The First Avenger (2011),Action|Adventure|Sci-Fi|Thriller|War,Captain America The First Avenger 2011
25071,0.224299,0.017589,12.752222,122920,Captain America: Civil War (2016),Action|Sci-Fi|Thriller,Captain America Civil War 2016
25058,0.186916,0.015015,12.448598,122892,Avengers: Age of Ultron (2015),Action|Adventure|Sci-Fi,Avengers Age of Ultron 2015
16527,0.186916,0.017160,10.892523,87232,X-Men: First Class (2011),Action|Adventure|Sci-Fi|Thriller|War,XMen First Class 2011


In [33]:
def find_similar_movies(movie_id):
    similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()
    similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

    similar_user_recs = similar_user_recs[similar_user_recs > .10]
    all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]
    all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())
    rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]

    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
    rec_percentages = rec_percentages.sort_values("score", ascending=False)
    return rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")[["score", "title", "genres"]]

In [34]:
import ipywidgets as widgets
from IPython.display import display

movie_name_input = widgets.Text(
    value='Toy Story',
    description='Movie Title:',
    disabled=False
)
recommendation_list = widgets.Output()

def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            results = search(title)
            movie_id = results.iloc[0]["movieId"]
            display(find_similar_movies(movie_id))

movie_name_input.observe(on_type, names='value')

display(movie_name_input, recommendation_list)

Text(value='Toy Story', description='Movie Title:')

Output()